In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import os
import keras

class DataRetrieval():
    """CLASS:DataRetrieval"""
    def __init__(self, data_dir):
        self.data_dir = data_dir
    
    def get_train_data(self, augmentation=True, img_input=(256,256), batch_size=16):
        if augmentation:
            datagen = ImageDataGenerator(
                featurewise_center=False,
                samplewise_center=False,
                featurewise_std_normalization=False,
                samplewise_std_normalization=False, 
                rotation_range=5,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.,
                zoom_range=0.2,
                fill_mode='nearest',
                horizontal_flip=True,
                vertical_flip=False,
                rescale=1./255
            )
        else:
            datagen = ImageDataGenerator(
                featurewise_center=False,
                samplewise_center=False,
                featurewise_std_normalization=False,
                samplewise_std_normalization=False,
            )
        return datagen.flow_from_directory(
            self.data_dir+os.sep+'train',
            target_size=img_input,
            batch_size=batch_size,
            class_mode='categorical'
        )

    def get_validation_data(self, img_input=(256,256), batch_size=16):
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            rescale=1./255
        )
        return datagen.flow_from_directory(
            self.data_dir+os.sep+'test',
            target_size=img_input,
            batch_size=batch_size,
            class_mode='categorical'
        )

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

def RoJoPow(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    # -- FIRST BLOCK -- #
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(filters=32, kernel_size=5, strides=2, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- SECOND BLOCK -- #
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,32))(y)
    y = layers.Dense(units=8,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=32,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- THIRD BLOCK -- #
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,64))(y)
    y = layers.Dense(units=16,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=64,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- FOURTH BLOCK -- #
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,64))(y)
    y = layers.Dense(units=16,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=64,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- FIFTH BLOCK -- #
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,128))(y)
    y = layers.Dense(units=32,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=128,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- SIXTH BLOCK -- #
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,128))(y)
    y = layers.Dense(units=32,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=128,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- DECISION BLOCK -- #
    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,8))(y)
    y = layers.Dense(units=8,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=8,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
model = final()

model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
activation_46 (Activation)   (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
activation_47 (Activation)   (None, 128, 128, 32)      0   

In [ ]:
def RoJoPow2(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    # -- FIRST BLOCK -- #
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(filters=64, kernel_size=5, strides=2, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- SECOND BLOCK -- #
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,64))(y)
    y = layers.Dense(units=16,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=64,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- THIRD BLOCK -- #
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,128))(y)
    y = layers.Dense(units=32,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=128,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- FOURTH BLOCK -- #
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,128))(y)
    y = layers.Dense(units=32,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=128,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.MaxPooling2D(pool_size=2)(x)
    # -- DECISION BLOCK -- #
    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    y = layers.GlobalAveragePooling2D()(x)
    y = layers.Reshape((1,1,8))(y)
    y = layers.Dense(units=8,kernel_initializer='he_normal')(y)
    y = layers.Activation('relu')(y)
    y = layers.Dense(units=8,kernel_initializer='he_normal')(y)
    y = layers.Activation('sigmoid')(y)
    x = layers.multiply([x, y])
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def mkdir(dirname):

    if os.path.isdir(dirname) and os.path.exists(dirname):
        return

    if not os.path.isdir(dirname) and os.path.exists(dirname):
        logging.info(f"remove file {dirname} and run this script again")

    os.mkdir(dirname)

In [ ]:
import zipfile
import os

path = os.getcwd() + '/datasets'
mkdir(path)

with zipfile.ZipFile("MIT_split-20210124T132955Z-001.zip", 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
from keras import models
from keras import layers
from keras import optimizers


def otherC(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Dropout(dropout)(x)

    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def baselineA(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def baselineB(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def final(input_shape=(256,256,3), optimizer=optimizers.Adam(), dropout=0.5):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(inputs)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(pool_size=2)(x)

    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=1, padding='same')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dropout(0.5)(x)

    x = layers.Conv2D(filters=8, kernel_size=1, strides=1, padding='valid')(x)
    x = layers.Activation('relu')(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Activation('softmax')(x)

    model = models.Model(inputs, x)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random

from keras.utils import plot_model
from keras import optimizers
from keras import callbacks

#from utils.Data import DataRetrieval
#from models import *


# Parameters
batch_size = 16
epochs = 200
input_shape = (256,256,3)
learning_rate = 1e-3
data_dir = '/content/datasets/MIT_split'
work_dir = '/content/baselineB'
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

# Create model
optimizer = optimizers.Adam(lr=learning_rate)
model = final(input_shape=input_shape, optimizer=optimizer, dropout=0.6)

# Print model
model.summary()
plot_model(model, to_file=work_dir+os.sep+'structure.png', show_shapes=True, show_layer_names=True)

# Prepare data
DR = DataRetrieval(data_dir)
train_generator = DR.get_train_data(augmentation=True, batch_size=batch_size)
validation_generator = DR.get_validation_data(batch_size=batch_size)

# Train model
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.2, patience=10)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=(int(train_generator.n//batch_size)),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= (int(validation_generator.n//batch_size)),
    callbacks=[reduce_lr, early_stopping])

# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(work_dir+os.sep+'accuracy.jpg')
plt.close()
# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.savefig(work_dir+os.sep+'loss.jpg')
plt.close()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 128, 128, 32)      0     

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
117/117 [==============================] - 38s 255ms/step - loss: 1.7227 - accuracy: 0.3640 - val_loss: 2.9706 - val_accuracy: 0.0938
Epoch 2/200
117/117 [==============================] - 30s 255ms/step - loss: 1.4248 - accuracy: 0.5754 - val_loss: 3.2072 - val_accuracy: 0.0938
Epoch 3/200
117/117 [==============================] - 30s 253ms/step - loss: 1.2970 - accuracy: 0.6554 - val_loss: 2.0973 - val_accuracy: 0.2362
Epoch 4/200
117/117 [==============================] - 30s 254ms/step - loss: 1.2216 - accuracy: 0.6595 - val_loss: 1.4140 - val_accuracy: 0.4512
Epoch 5/200
117/117 [==============================] - 30s 255ms/step - loss: 1.1185 - accuracy: 0.7127 - val_loss: 1.0526 - val_accuracy: 0.6587
Epoch 6/200
117/117 [==============================] - 30s 255ms/step - loss: 1.0935 - accuracy: 0.7035 - val_loss: 0.9911 - val_accuracy: 0.7000
Epoch 7/200
117/117 [==============================] - 30s 253ms/step - loss: 1.0225 - accuracy: 0.7432 - val_loss: 0.8727 -

In [ ]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), os.path.join(path, '..')))
  
if __name__ == '__main__':
    zipf = zipfile.ZipFile('Final_results_108_epochs.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('/content/Final', zipf)
    zipf.close()